In [1]:
import torch
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import cv2
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset,random_split
from PIL import Image
import numpy as np
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-06-03 06:35:23.689395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 06:35:23.689488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 06:35:23.813089: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SpatialPyramidPooling(nn.Module):
    def __init__(self, pool_list):
        super(SpatialPyramidPooling, self).__init__()
        self.pool_list = pool_list

    def forward(self, x):
        # Get the batch size and number of channels
        batch_size, num_channels, _, _ = x.size()

        # Initialize the output list
        pooled_outputs = []

        # Iterate over each pooling size in the pool_list
        for pool_size in self.pool_list:
            # Perform max pooling with the specified pool size
            pooled = F.adaptive_max_pool2d(x, output_size=(pool_size, pool_size))
            # Reshape the pooled output to have shape (batch_size, num_channels, pool_size * pool_size)
            pooled = pooled.view(batch_size, num_channels, -1)
            # Append the pooled output to the output list
            pooled_outputs.append(pooled)

        # Concatenate the pooled outputs along the last dimension
        output = torch.cat(pooled_outputs, dim=2)

        # Reshape the output to have shape (batch_size, num_channels * sum(pool_list))
        output = output.view(batch_size, -1)

        return output


In [3]:
# Define the backbone model (ResNet or MobileNet)
backbone = models.resnet18(pretrained=True)  # You can change to other models like MobileNet
#backbone = models.mobilenet_v2(pretrained=True)

# Remove the fully connected layer at the end
backbone = nn.Sequential(*list(backbone.children())[:-2])
input_channels = 512

backbone = backbone.to(device)
for param in backbone.parameters():
    param.requires_grad=False

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 145MB/s]


In [4]:
print(backbone)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [5]:
# Define the basic CNN architecture to predict steering angle
class SteeringCNN(nn.Module):
    def __init__(self ,backbone,input_shape, hidden_size, output_size):
        super(SteeringCNN, self).__init__()
        self.backbone = backbone
        self.spp = SpatialPyramidPooling(pool_list=[1, 2, 4])
        self.fc1 = nn.Linear(input_shape* (1 + 4 + 16), hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.spp(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
class SteeringDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = self.read_data_file()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name, angle = self.data[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path)
        angle = angle * 3.14159265 / 180
        if self.transform:
            image = self.transform(image)
        return image, angle

    def read_data_file(self):
        data_file = os.path.join(self.root_dir, 'data.txt')
        with open(data_file, 'r') as file:
            lines = file.readlines()
            data = [(os.path.join(self.root_dir,line.split()[0]), float(line.split()[1])) for line in lines]
        return data

In [7]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize input images to match backbone input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

In [8]:
# Define hyperparameters
input_size = 512  # Output feature size of ResNet18
hidden_size = 256
output_size = 1
learning_rate = 0.001
batch_size = 32
num_epochs = 70

In [9]:
# Create dataset and dataloaders
dataset = SteeringDataset(root_dir = r"/kaggle/input/driving-dataset/driving_dataset", transform=transform)
train_size = int(0.8*len(dataset))
val_size = int(0.1*len(dataset))
test_size = len(dataset) - train_size - val_size
train_ds,val_ds,test_ds = random_split(dataset,[train_size,val_size,test_size])
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [10]:
# Create models
model = SteeringCNN(backbone,input_channels, hidden_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
print(model)

SteeringCNN(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [12]:
# Initialize the list to store training losses
training_losses = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for images, angles in tqdm(train_dataloader, desc=f'Epoch {epoch+1}', leave=False):
        images = images.to(device)
        angles = angles.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, angles.float())
        train_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Calculate average training loss
    train_loss /= len(train_dataloader)
    training_losses.append(train_loss)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, angles in tqdm(val_dataloader, desc=f'Validation Epoch {epoch+1}', leave=False):
            images = images.to(device)
            angles = angles.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, angles.float())
            val_loss += loss.item()
    
    # Calculate average validation loss
    val_loss /= len(val_dataloader)
    
    # Print epoch statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

Epoch [1/70], Train Loss: 5.9987, Val Loss: 0.2804


Epoch [2/70], Train Loss: 0.2951, Val Loss: 0.2811


Epoch [3/70], Train Loss: 0.2942, Val Loss: 0.2826


Epoch [4/70], Train Loss: 0.2943, Val Loss: 0.2804


Epoch [5/70], Train Loss: 0.2930, Val Loss: 0.2801


...

Epoch [66/70], Train Loss: 0.2930, Val Loss: 0.2802


Epoch [67/70], Train Loss: 0.2930, Val Loss: 0.2801


Epoch [68/70], Train Loss: 0.2930, Val Loss: 0.2802


Epoch [69/70], Train Loss: 0.2930, Val Loss: 0.2807


Epoch [70/70], Train Loss: 0.2931, Val Loss: 0.2803


In [13]:
torch.save(model.state_dict(), 'steering_model.pth')

In [14]:
# Testing
model.eval()
test_loss = 0.0
with torch.no_grad():
    for images, angles in tqdm(test_dataloader,desc=f'Epoch {epoch+1}', leave=False):
        images = images.to(device)
        angles = angles.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, angles.unsqueeze(1).float())

        test_loss += loss.item()
print(f"TestLoss: {test_loss:.4f}")

TestLoss: 40.6227
